In [14]:
import numpy as np
from scipy import spatial
# Открываем файл и считываем строки в массив, обрезая мусор и приводя к нижнему регистру
d = [] # массив для хранения предложений текста в виде строк (далее массива строк)
n = 0 # число строк в файле
with open('sentences.txt') as file:
    for line in file:
        n += 1
        d.append(line.strip().lower()) # считываем строчки файла, приводя их к нижнему регистру
#print(d[:6])

import re # импортируем библиотеку для работы с регулярными выражениями
for i in range(len(d)): # пробегаем каждый элемент и разбиваем его регуляркой
    d[i] = re.split('[^a-z]', d[i])
    
    d[i] = [el for el in d[i] if el != ''] # удаляем пустые строки с помощью list comprehension
print(d)

[['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process'], ['as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', 'it', 'can', 'be', 'also', 'used', 'to', 'concatenate', 'binary', 'files', 'where', 'it', 'will', 'just', 'concatenate', 'sequence', 'of', 'bytes'], ['a', 'common', 'interactive', 'use', 'of', 'cat', 'for', 'a', 'single', 'file', 'is', 'to', 'output', 'the', 'content', 'of', 'a', 'file', 'to', 'standard', 'output'], ['cats', 'can', 'hear', 'sounds', 'too', 'faint', 'or', 'too', 'high', 'in', 'frequency', 'for', 'human', 'ears', 'such', 'as', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals'], ['in', 'one', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin'], ['the', 'domesticated', 'cat', 'and', 'its', 'closest', 'wild', 'ancestor', 'are', 'both', 'diploid', 'organism

In [7]:
fr = dict() # словарь числа вхождений слов в тексте
for el_d in d: # бежим по внешнему массиву
    for el in el_d: # по внутреннему массиву
        if el not in fr:
            fr[el] = 1 # если слова не было, заводим его
        else: 
            fr[el] += 1 # если слово есть, прибавляем число вхождений
print(fr)

{'in': 11, 'comparison': 1, 'to': 14, 'dogs': 1, 'cats': 4, 'have': 2, 'not': 1, 'undergone': 1, 'major': 2, 'changes': 2, 'during': 1, 'the': 20, 'domestication': 1, 'process': 2, 'as': 6, 'cat': 10, 'simply': 2, 'catenates': 1, 'streams': 1, 'of': 19, 'bytes': 2, 'it': 4, 'can': 3, 'be': 3, 'also': 2, 'used': 1, 'concatenate': 2, 'binary': 1, 'files': 1, 'where': 2, 'will': 2, 'just': 3, 'sequence': 2, 'a': 13, 'common': 1, 'interactive': 1, 'use': 2, 'for': 4, 'single': 1, 'file': 5, 'is': 7, 'output': 3, 'content': 2, 'standard': 1, 'hear': 1, 'sounds': 1, 'too': 2, 'faint': 1, 'or': 3, 'high': 1, 'frequency': 1, 'human': 2, 'ears': 1, 'such': 2, 'those': 1, 'made': 1, 'by': 3, 'mice': 1, 'and': 14, 'other': 2, 'small': 2, 'animals': 1, 'one': 4, 'people': 1, 'deliberately': 1, 'tamed': 1, 'artificial': 1, 'selection': 1, 'they': 1, 'were': 2, 'useful': 1, 'predators': 1, 'vermin': 1, 'domesticated': 1, 'its': 2, 'closest': 1, 'wild': 1, 'ancestor': 1, 'are': 2, 'both': 1, 'diploid

In [8]:
print(len(fr))

254


In [9]:
#Создаем список со словами, чтобы порядок не менялся
words_list = list(fr)
print(words_list, len(words_list))

['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process', 'as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', 'it', 'can', 'be', 'also', 'used', 'concatenate', 'binary', 'files', 'where', 'will', 'just', 'sequence', 'a', 'common', 'interactive', 'use', 'for', 'single', 'file', 'is', 'output', 'content', 'standard', 'hear', 'sounds', 'too', 'faint', 'or', 'high', 'frequency', 'human', 'ears', 'such', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals', 'one', 'people', 'deliberately', 'tamed', 'artificial', 'selection', 'they', 'were', 'useful', 'predators', 'vermin', 'domesticated', 'its', 'closest', 'wild', 'ancestor', 'are', 'both', 'diploid', 'organisms', 'that', 'possess', 'chromosomes', 'roughly', 'genes', 'domestic', 'similar', 'size', 'members', 'genus', 'felis', 'typically', 'weighing', 'between', 'kg', 'lb', 'however', 'if', 'piped', 'redirected', 'unnecessary', 'with', 'named

In [26]:
#Создаем матрицу (Количество предложений)*(Количество уникальных слов)
# элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение
matrix = []
diff = len(words_list) # количество уникальных слов
for i in range(n):
    temp_list = []
    for j in range(diff):
        temp_list.append(d[i].count(words_list[j]))
    matrix.append(np.array(temp_list))


In [25]:
d[0].count(words_list[243])

0

In [27]:
print(matrix[:5])

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
     

In [22]:
print(words_list)

['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process', 'as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', 'it', 'can', 'be', 'also', 'used', 'concatenate', 'binary', 'files', 'where', 'will', 'just', 'sequence', 'a', 'common', 'interactive', 'use', 'for', 'single', 'file', 'is', 'output', 'content', 'standard', 'hear', 'sounds', 'too', 'faint', 'or', 'high', 'frequency', 'human', 'ears', 'such', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals', 'one', 'people', 'deliberately', 'tamed', 'artificial', 'selection', 'they', 'were', 'useful', 'predators', 'vermin', 'domesticated', 'its', 'closest', 'wild', 'ancestor', 'are', 'both', 'diploid', 'organisms', 'that', 'possess', 'chromosomes', 'roughly', 'genes', 'domestic', 'similar', 'size', 'members', 'genus', 'felis', 'typically', 'weighing', 'between', 'kg', 'lb', 'however', 'if', 'piped', 'redirected', 'unnecessary', 'with', 'named

In [31]:
# массив для хранения косинусных расстояний для первого и остальных предложений 
result = [] 
for i in range(n):
    result.append(spatial.distance.cosine(matrix[0], matrix[i]))
print(result, n)

[0.0, 0.9527544408738466, 0.8644738145642124, 0.8951715163278082, 0.7770887149698589, 0.9402385695332803, 0.7327387580875756, 0.9258750683338899, 0.8842724875284311, 0.9055088817476932, 0.8328165362273942, 0.8804771390665607, 0.8396432548525454, 0.8703592552895671, 0.8740118423302576, 0.9442721787424647, 0.8406361854220809, 0.956644501523794, 0.9442721787424647, 0.8885443574849294, 0.8427572744917122, 0.8250364469440588] 22


In [ ]:
4 6

In [ ]:
# Вывод индексов самых близких к первому предложений